<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2024notebooks/2024_0927charRNN_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [青空文庫](https://www.aozora.gr.jp/) を用いた RNN のデモ


In [ ]:
# https://www.aozora.gr.jp/cards/000258/files/50326_35772.html
txt = "武蔵の国のある村に茂作、巳之吉と云う二人の木こりがいた。この話のあった時分には、茂作は老人であった。そして、彼の年季奉公人であった巳之吉は、十八の少年であった。毎日、彼等は村から約二里離れた森へ一緒に出かけた。その森へ行く道に、越さねばならない大きな河がある。そして、渡し船がある。渡しのある処にたびたび、橋が架けられたが、その橋は洪水のあるたびごとに流された。河の溢れる時には、普通の橋では、その急流を防ぐ事はできない。　茂作と巳之吉はある大層寒い晩、帰り途で大吹雪に遇った。渡し場に着いた、渡し守は船を河の向う側に残したままで、帰った事が分った。泳がれるような日ではなかった。それで木こりは渡し守の小屋に避難した――避難処の見つかった事を僥倖に思いながら。小屋には火鉢はなかった。火をたくべき場処もなかった。窓のない一方口の、二畳敷の小屋であった。茂作と巳之吉は戸をしめて、蓑をきて、休息するために横になった。初めのうちはさほど寒いとも感じなかった。そして、嵐はじきに止むと思った。　老人はじきに眠りについた。しかし、少年巳之吉は長い間、目をさましていて、恐ろしい風や戸にあたる雪のたえない音を聴いていた。河はゴウゴウと鳴っていた。小屋は海上の和船のようにゆれて、ミシミシ音がした。恐ろしい大吹雪であった。空気は一刻一刻、寒くなって来た、そして、巳之吉は蓑の下でふるえていた。しかし、とうとう寒さにも拘らず、彼もまた寝込んだ。　彼は顔に夕立のように雪がかかるので眼がさめた。小屋の戸は無理押しに開かれていた。そして雪明かりで、部屋のうちに女、――全く白装束の女、――を見た。その女は茂作の上に屈んで、彼に彼女の息をふきかけていた、――そして彼女の息はあかるい白い煙のようであった。ほとんど同時に巳之吉の方へ振り向いて、彼の上に屈んだ。彼は叫ぼうとしたが何の音も発する事ができなかった。白衣の女は、彼の上に段々低く屈んで、しまいに彼女の顔はほとんど彼にふれるようになった、そして彼は――彼女の眼は恐ろしかったが――彼女が大層綺麗である事を見た。しばらく彼女は彼を見続けていた、――それから彼女は微笑した、そしてささやいた、――『私は今ひとりの人のように、あなたをしようかと思った。しかし、あなたを気の毒だと思わずにはいられない、――あなたは若いのだから。……あなたは美少年ね、巳之吉さん、もう私はあなたを害しはしません。しかし、もしあなたが今夜見た事を誰かに――あなたの母さんにでも――云ったら、私に分ります、そして私、あなたを殺します。……覚えていらっしゃい、私の云う事を』　そう云って、向き直って、彼女は戸口から出て行った。その時、彼は自分の動ける事を知って、飛び起きて、外を見た。しかし、女はどこにも見えなかった。そして、雪は小屋の中へ烈しく吹きつけていた。巳之吉は戸をしめて、それに木の棒をいくつか立てかけてそれを支えた。彼は風が戸を吹きとばしたのかと思ってみた、――彼はただ夢を見ていたかもしれないと思った。それで入口の雪あかりの閃きを、白い女の形と思い違いしたのかもしれないと思った。しかもそれもたしかではなかった。彼は茂作を呼んでみた。そして、老人が返事をしなかったので驚いた。彼は暗がりへ手をやって茂作の顔にさわってみた。そして、それが氷である事が分った。茂作は固くなって死んでいた。……"

In [ ]:
import numpy as np
# 表示精度桁数の設定
import numpy as np
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

chars = list(sorted(set([ch for ch in txt])))
n_vocab = len(chars)
chr2idx = { ch:i for i,ch in enumerate(chars) }
idx2chr = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
n_hid = 128 # size of hidden layer of neurons
n_seq = 25 # number of steps to unroll the RNN for
lr = 1e-1

# model parameters
Wxh = np.random.randn(n_hid, len(chars)) * 0.01 # input to hidden
Whh = np.random.randn(n_hid, n_hid) * 0.01 # hidden to hidden
Why = np.random.randn(len(chars), n_hid) * 0.01 # hidden to output
bh = np.zeros((n_hid, 1)) # hidden bias
by = np.zeros((n_vocab, 1)) # output bias

def train(inputs:np.array,
          targets:np.array,
          H_init:np.array):
    """
        inputs, targets are both list of integers.
        h_init is Hx1 array of initial hidden state
        returns the loss, gradients on model parameters, and last hidden state
    """
    X, H, Y, Prob = {}, {}, {}, {}
    H[-1] = np.copy(H_init)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        X[t] = np.zeros((n_vocab,1)) # encode in 1-of-k representation
        X[t][inputs[t]] = 1
        H[t] = np.tanh(np.dot(Wxh, X[t]) + np.dot(Whh, H[t-1]) + bh) # hidden state
        Y[t] = np.dot(Why, H[t]) + by # unnormalized log probabilities for next chars
        Prob[t] = np.exp(Y[t]) / np.sum(np.exp(Y[t])) # probabilities for next chars
        loss += -np.log(Prob[t][targets[t],0]) # softmax (cross-entropy loss)

    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dh_init = np.zeros_like(H[0])
    for t in reversed(range(len(inputs))):
        dY = np.copy(Prob[t])
        dY[targets[t]] -= 1 # backprop into y
        dWhy += np.dot(dY, H[t].T)
        dby += dY
        dh = np.dot(Why.T, dY) + dh_init # backprop into h
        dh_ = (1 - H[t] * H[t]) * dh # backprop through tanh nonlinearity
        dbh += dh_
        dWxh += np.dot(dh_, X[t].T)
        dWhh += np.dot(dh_, H[t-1].T)
        dh_next = np.dot(Whh.T, dh_)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, H[len(inputs)-1]


def sample(H:np.array,
           seed_idx:int,
           n:int):
    """
        sample a sequence of integers from the model
        h is memory state, seed_idx is seed letter for first time step
    """
    X = np.zeros((n_vocab, 1))
    X[seed_idx] = 1
    ids = []
    for t in range(n):
        H = np.tanh(np.dot(Wxh, X) + np.dot(Whh, H) + bh)
        Y = np.dot(Why, H) + by
        Prob = np.exp(Y) / np.sum(np.exp(Y))
        _idx = np.random.choice(range(n_vocab), p=Prob.ravel())
        X = np.zeros((n_vocab, 1))
        X[_idx] = 1
        ids.append(_idx)
    return ids

In [ ]:
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/n_vocab) * n_seq # loss at iteration 0
n, p = 0, 0

epochs = 30000
for n in range(epochs):
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+n_seq+1 >= len(txt) or n == 0:
        h_init = np.zeros((n_hid,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [chr2idx[ch] for ch in txt[p:p+n_seq]]
    targets = [chr2idx[ch] for ch in txt[p+1:p+n_seq+1]]

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = train(inputs, targets, h_init)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # sample from the model now and then
    if n % 100 == 0:
        print(f'反復回数: {n:6d}, 損失: {smooth_loss:8.3f}', end=": ")

        sample_ids = sample(h_init, inputs[0], 50)
        out_txt = ''.join(idx2chr[idx] for idx in sample_ids)
        print(f'サンプリング： {out_txt}')

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -lr * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += n_seq # move data pointer
    #n += 1 # iteration counter


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class rnn(torch.nn.Module):
    def __init__(self,
                 rnn_type:str='LSTM',
                 ntoken:int=None,
                 ninp:int=None,
                 nhid:int=None,
                 nlayers:int=None,
                 dropout:float=0.5,
                 tie_weights:bool=False):
        super().__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError as e:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""") from e
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)